In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from osgeo import gdal
import os


In [ ]:
csv1 = pd.read_csv('/content/drive/MyDrive/saocom.tif/book3.csv')
csv1.columns =['row','column','hh_december','hh_january','hh_february','hh_march','hv_december','hv_january','hv_february','hv_march','vh_december','vh_january','vh_february','vh_march','vv_december','vv_january','vv_february','vv_march']

In [ ]:
csv2 =csv1.drop(['row','column'],axis=1)
csv2

,hh_december,hh_january,hh_february,hh_march,hv_december,hv_january,hv_february,hv_march,vh_december,vh_january,vh_february,vh_march,vv_december,vv_january,vv_february,vv_march
0,0.195508,0.239521,0.227744,0.296569,0.097196,0.117516,0.084969,0.114690,0.106125,0.121826,0.093729,0.109364,0.185899,0.228060,0.173006,0.219308
1,0.239030,0.262389,0.234537,0.249500,0.082254,0.072966,0.094531,0.061347,0.064763,0.064763,0.076051,0.066819,0.204126,0.263012,0.235383,0.223427
2,0.371814,0.523594,0.450607,0.669142,0.073193,0.070447,0.070134,0.111064,0.069602,0.057231,0.067349,0.097908,0.405631,0.608025,0.390667,0.416309
3,0.457768,0.452117,0.425122,0.391990,0.161751,0.188025,0.153317,0.131597,0.164665,0.196644,0.141966,0.114373,0.337416,0.419156,0.332781,0.308535
4,0.568742,0.501411,0.644075,0.586924,0.052890,0.054584,0.073803,0.111112,0.060033,0.055781,0.075902,0.106666,0.584701,0.387321,0.285475,0.310903
5,0.641273,1.508002,0.729230,0.354767,0.113730,0.104104,0.104934,0.086616,0.091898,0.101905,0.103762,0.082940,0.619858,0.804385,0.497720,0.294507
6,0.743260,0.772886,0.711989,0.306842,0.065088,0.105574,0.111508,0.094249,0.071902,0.080185,0.087130,0.101502,0.503803,1.053828,0.722523,0.333404
7,0.871315,0.664569,0.563332,0.770834,0.094249,0.113282,0.101531,0.176092,0.050534,0.075851,0.099413,0.169150,0.392525,0.950200,0.490379,0.629296
8,0.933570,0.971671,0.905188,0.629894,0.125794,0.106341,0.085510,0.118261,0.137867,0.106864,0.107787,0.118483,0.553940,0.519739,0.586025,0.462088
9,0.159274,0.212751,0.259026,0.396837,0.026422,0.041073,0.060778,0.090777,0.025980,0.036459,0.069687,0.083452,0.164145,0.200432,0.149254,0.197086


In [ ]:
v = csv2.values.tolist()
v[1]
v2 = np.array(v[1],dtype = np.float)
v2.shape

(16,)

In [ ]:
def dp(df7,df8,m,n):
    
  
    
    

    #N = df7.shape
    #M = df8.shape
    
    # Initialize the cost matrix
    cost_mat = np.zeros((17, 17))
    for i in range(1, 17):
        cost_mat[i, 0] = np.inf
    for i in range(1, 17):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((16, 16))
    for i in range(16):
        for j in range(16):
            penalty = [
                cost_mat[i, j],      # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = abs(df7[i]-df8[j]) + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = 16 - 1
    j = 16 - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))
        

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    m.append(cost_mat)
    n.append(path[::-1])
    return m,n

In [ ]:
s =csv2.values.tolist()

m =[]
n = []

for k in range(41):
  list = np.array(s[k],dtype = np.float)
  
  dp(v2,list,m,n)
  print(m)
  print(n)

Streaming output truncated to the last 5000 lines.
        0.6071028 , 0.63330363, 0.78955037, 1.12270235, 1.38717546,
        1.70137513],
       [2.30825324, 2.31184588, 2.37900297, 2.51019389, 0.86753826,
        0.8623593 , 0.72903959, 0.71082799, 0.78035252, 0.79340523,
        0.73881491, 0.71261408, 0.66511115, 0.87018746, 1.00658491,
        1.19270891],
       [2.35918955, 2.36278219, 2.42993928, 2.56113021, 1.08695498,
        1.08112865, 0.91968867, 0.88326547, 0.93457841, 1.00251757,
        0.92941293, 0.87701127, 0.69218953, 0.81130156, 0.88881311,
        1.0160512 ],
       [2.437755  , 2.44134764, 2.50850473, 2.63969566, 1.27874257,
        1.27226887, 1.08270861, 1.02807382, 1.07938675, 1.12911432,
        1.09238181, 1.01377932, 0.69274029, 0.86600907, 0.91644225,
        1.04368034],
       [2.52827677, 2.53186941, 2.5990265 , 2.73021742, 1.45857384,
        1.45145277, 1.23377223, 1.16092584, 1.21223877, 1.26196634,
        1.24339437, 1.13859105, 0.70524737, 0.878

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
h = []
for i in m:
  k =(i[-1,-1])
  h.append(k)
  

In [ ]:
h

[0.507084487,
 0.0,
 2.012664368,
 1.8548138810000003,
 2.07833082,
 3.719462173,
 3.3652044320000005,
 3.603508648,
 3.9859502229999997,
 0.6915142559999999,
 1.1035293590000002,
 1.0684071139999998,
 3.2558694040000002,
 1.0070445439999998,
 3.5682344509999995,
 2.079865994,
 2.296355127,
 3.000256883999999,
 1.4202602569999998,
 0.731887357,
 0.5934430049999999,
 0.383745173,
 1.0832657960000003,
 1.9390452860000003,
 1.8312888090000001,
 3.2583332460000003,
 2.646960406,
 2.3395582720000005,
 3.158608049,
 2.4941760029999998,
 2.179569181,
 3.80884918,
 2.8913938500000005,
 3.91215753,
 4.313307611,
 2.4010349770000006,
 2.0129601900000003,
 3.634115302,
 4.303995334,
 3.9613899899999994,
 3.3041179659999997]

In [ ]:
csv3 = csv1[['row','column']]
csv3

s = csv3['row'].tolist()
t = csv3['column'].tolist()



In [ ]:
s[1]

359

In [ ]:
matrix = np.zeros([721,1231])
matrix

for i in range(41):
  matrix[s[i],t[i]]= h[i]


  


ValueError: ignored